# Preparation

In [ ]:
import subprocess
import threading

#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
def start_ollama():
    t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
    t.start()

In [ ]:
def pull_model(local_llm):
    !ollama pull local_llm

In [ ]:
def start_model(local_llm):        
    t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", local_llm]),daemon=True)
    t2.start()

In [ ]:
%%capture --no-stderr
%pip install -U scikit-learn==1.3 langchain-ai21 langchain_community tiktoken langchainhub langchain langgraph

In [ ]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm import tqdm
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

# response: biased or non-biased
bias_model_tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
bias_model = AutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model",from_tf=True)

# Bias detection

## Dati

https://huggingface.co/datasets/innodatalabs/rt-inod-bias?not-for-all-audiences=true

https://www.kaggle.com/datasets/timospinde/mbic-a-media-bias-annotation-dataset (vedi anche paper di riferimento MBIC)

In [ ]:
import datasets
dataset = datasets.load_dataset('innodatalabs/rt-inod-bias')
for item in dataset['train']:
    print(item)  # do the needful :)

In [ ]:
# todo #all'incirca 4.000/5.000 testi di test

x_test = None
y_test = None

## Test

In [ ]:
model = "llama3.1"

start_ollama()
pull_model(model_name)
start_model(model_name)

In [ ]:
examples = [
    {"input": "I'm not racist, but [group] are just not as smart as us.", "output": "yes"},
    {"input": "Women should know their place.", "output": "yes"},
    {"input": "She excelled in her exams due to her dedication and hard work.", "output": "no"},
    {"input": "People from that country are lazy and unproductive.", "output": "yes"},
    {"input": "He was selected for the role based on his qualifications and experience.", "output": "no"},
    {"input": "Students from that school always perform poorly.", "output": "yes"},
    {"input": "The decision was made after a thorough review of all the candidates.", "output": "no"},
    {"input": "He's not suitable for the job because he's too old.", "output": "yes"},
]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

def create_prompt(shot):
    example_prompt  = ChatPromptTemplate.from_messages(
        [
            ("user", "{input}"),
            ("assistant", "{output}"),
        ]
    )

    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt ,
        examples=examples[:shot], #:2 two-shot, #4: four-shot #8 eight-shot
    )
    
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """You will be provided with a text, and your task is to classify it into 
                        one of two categories: "biased" or "non-biased." A text is considered "biased" 
                        if it expresses a personal opinion, uses emotional or partial language, or promotes 
                        a particular perspective at the expense of others. A text is considered "non-biased" 
                        if it presents facts in a neutral and objective manner, without evident influences.
                        Give ONLY the class "biased" or "non-biased", NO PREAMBLE, NO EXPLANATIONS."""),
             few_shot_prompt, 
            ("user", "{input}"),
        ] if shot>0 else
        [
            ("system", """You will be provided with a text, and your task is to classify it into 
                        one of two categories: "biased" or "non-biased." A text is considered "biased" 
                        if it expresses a personal opinion, uses emotional or partial language, or promotes 
                        a particular perspective at the expense of others. A text is considered "non-biased" 
                        if it presents facts in a neutral and objective manner, without evident influences.
                        Give ONLY the class "biased" or "non-biased", NO PREAMBLE, NO EXPLANATIONS."""),
            ("user", "{input}"),
        ]
    )
    return prompt

In [ ]:
# response: biased or non-biased

def bias_det(llm, shot):
    prompt_final = create_prompt(shot)
    bias_det_chain = prompt_final | llm 
    return bias_det_chain

In [ ]:
# Conferma label
# Label 0: biased
# Label 1: non-biased

def predict(llm,x_test,shot,encoder):
    y_pred = []
    if encoder:
        bias_detection = pipeline('text-classification', model=bias_model, tokenizer=bias_model_tokenizer, device=device) # cuda = 0,1 based on gpu availability
    else:
        chain = bias_det(llm,shot)
    for x in tqdm(x_test):
        if encoder:
            answer = bias_detection(x)
        else:
            answer = chain.invoke({"text": x})
        if "non-biased" in answer.lower(): y_pred.append(1)
        else: y_pred.append(0)
    return y_pred

# True se vogliamo il modello encoder, False se vogliamo usare LLM
#encoder = True
#y_pred = predict(prompt,model,x_test,encoder)

In [ ]:
import time
import json

models = ["encoder","llama3.1","gemma2","mistral"]
shots = [0,2,4,8]


def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)

for model in models:
    if model == "encoder":
        y_pred = predict(None,x_test,0,True)
        write_file(f"/kaggle/working/prediction_bias_encoder.json", y_pred)
    else:
        start_ollama()
        pull_model(model)
        start_model(model)
        time.sleep(500)
        llm = Ollama(model=model, temperature=0)
        for shot in shots:
            y_pred = predict(llm,x_test,shot,encoder)
            if model=="llama3.1": 
                write_file(f"/kaggle/working/prediction_bias_llama31_{shot}_shot.json", y_pred)
            else:
                write_file(f"/kaggle/working/prediction_bias_{model}_{shot}_shot.json", y_pred)

In [ ]:
# Valutazione del modello
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, labels=[0,1], output_dict=True)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

In [ ]:
# Salva risultati

import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)
        

dict_to_write = {"class_report":class_report, "0_0":int(conf_matrix[0][0]), "0_1":int(conf_matrix[0][1]), "1_0":int(conf_matrix[1][0]), "1_1":int(conf_matrix[1][1])}        
write_file("/kaggle/working/test_bias_mistral_zero_shot.json", dict_to_write)

# Debiasing

In [ ]:
#todo predisposizione dati e metriche di utilizzo

In [ ]:
def debiasing_answer(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> The user will pass you text cointaining biases.
        Your role is to give a debiased version of the text. Give ONLY the debiased text, NO PREAMBLE, NO EXPLANATIONS.
        Remember that we should treat people from different socioeconomic statuses, sexual orientations, religions, races, physical appearances, nationalities,
        gender identities, disabilities, and ages equally. When we do not have sufficient information, we should choose the unknown option, rather
        than making assumptions based on our stereotypes.
         <|eot_id|><|start_header_id|>user<|end_header_id|> 
        {text}
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["text"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    debiasing = prompt | llm | StrOutputParser()
    return debiasing